In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [18]:
beijing_aq = pd.read_csv('beijing_aq_preprocessed.csv.gz')
beijing_aq.head(2)

,time,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2014-05-01 00:00:00,1001,138.0,159.4,56.3,0.9,50.8,17.2
1,2014-05-01 00:00:00,1002,89.0,132.9,30.5,0.8,96.5,7.6


In [19]:
beijing_met = pd.read_csv('beijing_met_preprocessed.csv.gz')
beijing_met.head(2)

,time,station_id,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,1001,0.0,20.0,56.0,7.92,13.0
1,2014-05-01 00:00:00,1002,0.0,20.0,56.0,7.92,13.0


In [7]:
# beijing_met['time'] = pd.to_datetime(beijing_met['time'])
# beijing_aq['time'] = pd.to_datetime(beijing_aq['time'])
# beijing_aq_plus_met2 = beijing_aq_plus_met.sort_values(by=['time', 'station_id'])
# beijing_aq_plus_met2.head(2)

### Asserting number of data entries per station

In [59]:
# for station in beijing_aq_plus_met2.station_id.unique():
#     tmp_df = beijing_aq_plus_met2[beijing_aq_plus_met2.station_id==station]
#     assert tmp_df.shape == (8760, 13)
methods = ['linear','slinear','quadratic','cubic','spline','akima']

### Method 1 : Avg of RMSE

In [116]:
for cols in ['temperature','humidity','wind_speed']:
    
    alinear_loss,aslinear_loss,aquadratic_loss,acubic_loss,aspline_loss,aakima_loss = [],[],[],[],[],[]
    
    for station in tqdm(beijing_met.station_id.unique()):
        
        linear_loss,slinear_loss,quadratic_loss,cubic_loss,spline_loss,akima_loss = [],[],[],[],[],[]
        tmp_df = beijing_met[beijing_met.station_id==station].reset_index(drop=True)
        assert tmp_df.shape == (8760, len(beijing_met.columns))
        
        tmp_df = tmp_df.iloc[:,[beijing_met.columns.get_loc('time'),beijing_met.columns.get_loc(cols)]]
        
        dates = pd.to_datetime([datetime.fromisoformat(tmp_df.iloc[i,tmp_df.columns.get_loc('time')]) for i in range(len(tmp_df))])
        time_differences = ((dates - dates[0]).total_seconds() // 3600)
        tmp_df['delta_t'] = time_differences
        tmp_df = tmp_df.drop(columns='time')
        
        tmp_df_notmissing = tmp_df.dropna(subset=[cols]).reset_index(drop=True)
        indices = ShuffleSplit(n_splits=5, test_size=.2, random_state=0).split(tmp_df_notmissing)
        
        for i in indices:
            train_indices = list(i[0])
            test_indices = list(i[1])
            train_indices.sort(),test_indices.sort()
   
            xtest = tmp_df_notmissing.iloc[test_indices,:]
            xtest = xtest.set_index('delta_t')
            xtrain = tmp_df_notmissing.copy()
            xtrain.iloc[test_indices,xtrain.columns.get_loc(cols)] = np.nan
            xtrain = xtrain.set_index('delta_t')
            
            interpolate_linear = xtrain.interpolate(method='linear')
            interpolate_linear = interpolate_linear.ffill()
            interpolate_linear = interpolate_linear.bfill()
            
            interpolate_slinear = xtrain.interpolate(method='slinear')
            interpolate_slinear = interpolate_slinear.ffill()
            interpolate_slinear = interpolate_slinear.bfill()
            
            interpolate_quadratic = xtrain.interpolate(method='quadratic')
            interpolate_quadratic = interpolate_quadratic.ffill()
            interpolate_quadratic = interpolate_quadratic.bfill()
            
            interpolate_cubic = xtrain.interpolate(method='cubic')
            interpolate_cubic = interpolate_cubic.ffill()
            interpolate_cubic = interpolate_cubic.bfill()
            
            interpolate_spline = xtrain.interpolate(method='spline',order=3)
            interpolate_spline = interpolate_spline.ffill()
            interpolate_spline = interpolate_spline.bfill()
            
            interpolate_akima = xtrain.interpolate(method='akima')
            interpolate_akima = interpolate_akima.ffill()
            interpolate_akima = interpolate_akima.bfill()

            mse_linear = mean_squared_error(xtest,interpolate_linear.iloc[test_indices,0],squared=False)
            mse_slinear = mean_squared_error(xtest,interpolate_slinear.iloc[test_indices,0],squared=False)
            mse_quadratic = mean_squared_error(xtest,interpolate_quadratic.iloc[test_indices,0],squared=False)
            mse_cubic = mean_squared_error(xtest,interpolate_cubic.iloc[test_indices,0],squared=False)
            mse_spline = mean_squared_error(xtest,interpolate_spline.iloc[test_indices,0],squared=False)
            mse_akima = mean_squared_error(xtest,interpolate_akima.iloc[test_indices,0],squared=False)
            
            linear_loss.append(mse_linear),slinear_loss.append(mse_slinear),quadratic_loss.append(mse_quadratic),cubic_loss.append(mse_cubic)
            spline_loss.append(mse_spline),akima_loss.append(mse_akima)
            
        alinear_loss.append(np.mean(mse_linear)),aslinear_loss.append(np.mean(mse_slinear)),aquadratic_loss.append(np.mean(mse_quadratic)),acubic_loss.append(np.mean(mse_cubic))
        aspline_loss.append(np.mean(mse_spline)),aakima_loss.append(np.mean(mse_akima))
        
    print(np.mean(alinear_loss),np.mean(aslinear_loss),np.mean(aquadratic_loss),np.mean(acubic_loss),np.mean(aspline_loss),np.mean(aakima_loss))
    method = methods[np.argmin(np.array([np.mean(alinear_loss),np.mean(aslinear_loss),np.mean(aquadratic_loss),np.mean(acubic_loss),np.mean(aspline_loss),np.mean(aakima_loss)]))]
    print(cols,method)

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

1.2880549335463727 1.2553435394418677 1.463775516240924 1.4972760216294518 1.553201714525396 1.2699705921441573
temperature slinear


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

7.355662841813643 7.278641911844814 8.984151486149889 9.233955428384323 9.17568295569436 7.3832641802767025
humidity slinear


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [02:10<00:00,  4.20s/it]

3.3075697940423363 3.3022554011973266 4.136850466097549 4.295506203864383 4.116862366070366 3.4603106226575253
wind_speed slinear


In [117]:
for cols in ['PM25_Concentration']:
    
    alinear_loss,aslinear_loss,aquadratic_loss,acubic_loss,aspline_loss,aakima_loss = [],[],[],[],[],[]
    
    for station in tqdm(beijing_aq.station_id.unique()):
        
        linear_loss,slinear_loss,quadratic_loss,cubic_loss,spline_loss,akima_loss = [],[],[],[],[],[]
        tmp_df = beijing_aq[beijing_aq.station_id==station].reset_index(drop=True)
        assert tmp_df.shape == (8760, len(beijing_aq.columns))
        
        tmp_df = tmp_df.iloc[:,[beijing_aq.columns.get_loc('time'),beijing_aq.columns.get_loc(cols)]]
        
        dates = pd.to_datetime([datetime.fromisoformat(tmp_df.iloc[i,tmp_df.columns.get_loc('time')]) for i in range(len(tmp_df))])
        time_differences = ((dates - dates[0]).total_seconds() // 3600)
        tmp_df['delta_t'] = time_differences
        tmp_df = tmp_df.drop(columns='time')
        
        tmp_df_notmissing = tmp_df.dropna(subset=[cols]).reset_index(drop=True)
        indices = ShuffleSplit(n_splits=5, test_size=.2, random_state=0).split(tmp_df_notmissing)
        
        for i in indices:
            train_indices = list(i[0])
            test_indices = list(i[1])
            train_indices.sort(),test_indices.sort()
   
            xtest = tmp_df_notmissing.iloc[test_indices,:]
            xtest = xtest.set_index('delta_t')
            xtrain = tmp_df_notmissing.copy()
            xtrain.iloc[test_indices,xtrain.columns.get_loc(cols)] = np.nan
            xtrain = xtrain.set_index('delta_t')
            
            interpolate_linear = xtrain.interpolate(method='linear')
            interpolate_linear = interpolate_linear.ffill()
            interpolate_linear = interpolate_linear.bfill()
            
            interpolate_slinear = xtrain.interpolate(method='slinear')
            interpolate_slinear = interpolate_slinear.ffill()
            interpolate_slinear = interpolate_slinear.bfill()
            
            interpolate_quadratic = xtrain.interpolate(method='quadratic')
            interpolate_quadratic = interpolate_quadratic.ffill()
            interpolate_quadratic = interpolate_quadratic.bfill()
            
            interpolate_cubic = xtrain.interpolate(method='cubic')
            interpolate_cubic = interpolate_cubic.ffill()
            interpolate_cubic = interpolate_cubic.bfill()
            
            interpolate_spline = xtrain.interpolate(method='spline',order=3)
            interpolate_spline = interpolate_spline.ffill()
            interpolate_spline = interpolate_spline.bfill()
            
            interpolate_akima = xtrain.interpolate(method='akima')
            interpolate_akima = interpolate_akima.ffill()
            interpolate_akima = interpolate_akima.bfill()

            mse_linear = mean_squared_error(xtest,interpolate_linear.iloc[test_indices,0],squared=False)
            mse_slinear = mean_squared_error(xtest,interpolate_slinear.iloc[test_indices,0],squared=False)
            mse_quadratic = mean_squared_error(xtest,interpolate_quadratic.iloc[test_indices,0],squared=False)
            mse_cubic = mean_squared_error(xtest,interpolate_cubic.iloc[test_indices,0],squared=False)
            mse_spline = mean_squared_error(xtest,interpolate_spline.iloc[test_indices,0],squared=False)
            mse_akima = mean_squared_error(xtest,interpolate_akima.iloc[test_indices,0],squared=False)
            
            linear_loss.append(mse_linear),slinear_loss.append(mse_slinear),quadratic_loss.append(mse_quadratic),cubic_loss.append(mse_cubic)
            spline_loss.append(mse_spline),akima_loss.append(mse_akima)
            
        alinear_loss.append(np.mean(mse_linear)),aslinear_loss.append(np.mean(mse_slinear)),aquadratic_loss.append(np.mean(mse_quadratic)),acubic_loss.append(np.mean(mse_cubic))
        aspline_loss.append(np.mean(mse_spline)),aakima_loss.append(np.mean(mse_akima))
        
    print(np.mean(alinear_loss),np.mean(aslinear_loss),np.mean(aquadratic_loss),np.mean(acubic_loss),np.mean(aspline_loss),np.mean(aakima_loss))
    method = methods[np.argmin(np.array([np.mean(alinear_loss),np.mean(aslinear_loss),np.mean(aquadratic_loss),np.mean(acubic_loss),np.mean(aspline_loss),np.mean(aakima_loss)]))]
    print(cols,method)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [06:01<00:00, 11.67s/it]

16.650657927097026 16.750067359559136 19.745632065530536 20.009703052003655 20.74208660150826 16.546232752415225
PM25_Concentration akima


### Method 2 : Flatten and take RMSE

In [115]:
for cols in ['temperature','humidity','wind_speed']:
    
    testing,ind = [],[]
    linear_loss,slinear_loss,quadratic_loss,cubic_loss,spline_loss,akima_loss = [],[],[],[],[],[]
    
    for station in tqdm(beijing_met.station_id.unique()):
        
        
        tmp_df = beijing_met[beijing_met.station_id==station].reset_index(drop=True)
        assert tmp_df.shape == (8760, len(beijing_met.columns))
        
        tmp_df = tmp_df.iloc[:,[beijing_met.columns.get_loc('time'),beijing_met.columns.get_loc(cols)]]
        
        dates = pd.to_datetime([datetime.fromisoformat(tmp_df.iloc[i,tmp_df.columns.get_loc('time')]) for i in range(len(tmp_df))])
        time_differences = ((dates - dates[0]).total_seconds() // 3600)
        tmp_df['delta_t'] = time_differences
        tmp_df = tmp_df.drop(columns='time')
        
        tmp_df_notmissing = tmp_df.dropna(subset=[cols]).reset_index(drop=True)
        indices = ShuffleSplit(n_splits=5, test_size=.2, random_state=0).split(tmp_df_notmissing)
        
        for i in indices:
            train_indices = list(i[0])
            test_indices = list(i[1])
            train_indices.sort(),test_indices.sort()
   
            xtest = tmp_df_notmissing.iloc[test_indices,:]
            xtest = xtest.set_index('delta_t')
            xtrain = tmp_df_notmissing.copy()
            xtrain.iloc[test_indices,xtrain.columns.get_loc(cols)] = np.nan
            xtrain = xtrain.set_index('delta_t')
            
            interpolate_linear = xtrain.interpolate(method='linear')
            interpolate_linear = interpolate_linear.ffill()
            interpolate_linear = interpolate_linear.bfill()
            
            interpolate_slinear = xtrain.interpolate(method='slinear')
            interpolate_slinear = interpolate_slinear.ffill()
            interpolate_slinear = interpolate_slinear.bfill()
            
            interpolate_quadratic = xtrain.interpolate(method='quadratic')
            interpolate_quadratic = interpolate_quadratic.ffill()
            interpolate_quadratic = interpolate_quadratic.bfill()
            
            interpolate_cubic = xtrain.interpolate(method='cubic')
            interpolate_cubic = interpolate_cubic.ffill()
            interpolate_cubic = interpolate_cubic.bfill()
            
            interpolate_spline = xtrain.interpolate(method='spline',order=3)
            interpolate_spline = interpolate_spline.ffill()
            interpolate_spline = interpolate_spline.bfill()
            
            interpolate_akima = xtrain.interpolate(method='akima')
            interpolate_akima = interpolate_akima.ffill()
            interpolate_akima = interpolate_akima.bfill()

            
            linear_loss.append(interpolate_linear.iloc[test_indices,0]),slinear_loss.append(interpolate_slinear.iloc[test_indices,0]),quadratic_loss.append(interpolate_quadratic.iloc[test_indices,0]),cubic_loss.append(interpolate_cubic.iloc[test_indices,0])
            spline_loss.append(interpolate_spline.iloc[test_indices,0]),akima_loss.append(interpolate_akima.iloc[test_indices,0])
            testing.append(xtest)
    
    interpolate_linear = list(np.concatenate(linear_loss).flat)
    interpolate_slinear = list(np.concatenate(slinear_loss).flat)
    interpolate_quadratic = list(np.concatenate(quadratic_loss).flat)
    interpolate_cubic = list(np.concatenate(cubic_loss).flat)
    interpolate_spline = list(np.concatenate(spline_loss).flat)
    interpolate_akima = list(np.concatenate(akima_loss).flat)
    
    xtest = list(np.concatenate(testing).flat)
#     print(xtest)
    mse_linear = mean_squared_error(xtest,interpolate_linear,squared=False)
    mse_slinear = mean_squared_error(xtest,interpolate_slinear,squared=False)
    mse_quadratic = mean_squared_error(xtest,interpolate_quadratic,squared=False)
    mse_cubic = mean_squared_error(xtest,interpolate_cubic,squared=False)
    mse_spline = mean_squared_error(xtest,interpolate_spline,squared=False)
    mse_akima = mean_squared_error(xtest,interpolate_akima,squared=False)
            
    method = methods[np.argmin(np.array([mse_linear,mse_slinear,mse_quadratic,mse_cubic,mse_spline,mse_akima]))]
    print(cols,method)

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

temperature akima


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

humidity slinear


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [03:16<00:00,  6.33s/it]

wind_speed slinear


In [118]:
for cols in ['PM25_Concentration']:
    
    testing,ind = [],[]
    linear_loss,slinear_loss,quadratic_loss,cubic_loss,spline_loss,akima_loss = [],[],[],[],[],[]
    
    for station in tqdm(beijing_aq.station_id.unique()):
        
        
        tmp_df = beijing_aq[beijing_aq.station_id==station].reset_index(drop=True)
        assert tmp_df.shape == (8760, len(beijing_aq.columns))
        
        tmp_df = tmp_df.iloc[:,[beijing_aq.columns.get_loc('time'),beijing_aq.columns.get_loc(cols)]]
        
        dates = pd.to_datetime([datetime.fromisoformat(tmp_df.iloc[i,tmp_df.columns.get_loc('time')]) for i in range(len(tmp_df))])
        time_differences = ((dates - dates[0]).total_seconds() // 3600)
        tmp_df['delta_t'] = time_differences
        tmp_df = tmp_df.drop(columns='time')
        
        tmp_df_notmissing = tmp_df.dropna(subset=[cols]).reset_index(drop=True)
        indices = ShuffleSplit(n_splits=5, test_size=.2, random_state=0).split(tmp_df_notmissing)
        
        for i in indices:
            train_indices = list(i[0])
            test_indices = list(i[1])
            train_indices.sort(),test_indices.sort()
   
            xtest = tmp_df_notmissing.iloc[test_indices,:]
            xtest = xtest.set_index('delta_t')
            xtrain = tmp_df_notmissing.copy()
            xtrain.iloc[test_indices,xtrain.columns.get_loc(cols)] = np.nan
            xtrain = xtrain.set_index('delta_t')
            
            interpolate_linear = xtrain.interpolate(method='linear')
            interpolate_linear = interpolate_linear.ffill()
            interpolate_linear = interpolate_linear.bfill()
            
            interpolate_slinear = xtrain.interpolate(method='slinear')
            interpolate_slinear = interpolate_slinear.ffill()
            interpolate_slinear = interpolate_slinear.bfill()
            
            interpolate_quadratic = xtrain.interpolate(method='quadratic')
            interpolate_quadratic = interpolate_quadratic.ffill()
            interpolate_quadratic = interpolate_quadratic.bfill()
            
            interpolate_cubic = xtrain.interpolate(method='cubic')
            interpolate_cubic = interpolate_cubic.ffill()
            interpolate_cubic = interpolate_cubic.bfill()
            
            interpolate_spline = xtrain.interpolate(method='spline',order=3)
            interpolate_spline = interpolate_spline.ffill()
            interpolate_spline = interpolate_spline.bfill()
            
            interpolate_akima = xtrain.interpolate(method='akima')
            interpolate_akima = interpolate_akima.ffill()
            interpolate_akima = interpolate_akima.bfill()

            
            linear_loss.append(interpolate_linear.iloc[test_indices,0]),slinear_loss.append(interpolate_slinear.iloc[test_indices,0]),quadratic_loss.append(interpolate_quadratic.iloc[test_indices,0]),cubic_loss.append(interpolate_cubic.iloc[test_indices,0])
            spline_loss.append(interpolate_spline.iloc[test_indices,0]),akima_loss.append(interpolate_akima.iloc[test_indices,0])
            testing.append(xtest)
    
    interpolate_linear = list(np.concatenate(linear_loss).flat)
    interpolate_slinear = list(np.concatenate(slinear_loss).flat)
    interpolate_quadratic = list(np.concatenate(quadratic_loss).flat)
    interpolate_cubic = list(np.concatenate(cubic_loss).flat)
    interpolate_spline = list(np.concatenate(spline_loss).flat)
    interpolate_akima = list(np.concatenate(akima_loss).flat)
    
    xtest = list(np.concatenate(testing).flat)
#     print(xtest)
    mse_linear = mean_squared_error(xtest,interpolate_linear,squared=False)
    mse_slinear = mean_squared_error(xtest,interpolate_slinear,squared=False)
    mse_quadratic = mean_squared_error(xtest,interpolate_quadratic,squared=False)
    mse_cubic = mean_squared_error(xtest,interpolate_cubic,squared=False)
    mse_spline = mean_squared_error(xtest,interpolate_spline,squared=False)
    mse_akima = mean_squared_error(xtest,interpolate_akima,squared=False)
            
    method = methods[np.argmin(np.array([mse_linear,mse_slinear,mse_quadratic,mse_cubic,mse_spline,mse_akima]))]
    print(cols,method)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [05:53<00:00, 11.40s/it]

PM25_Concentration akima


In [108]:
station_wise_met_df,station_wise_aq_df = [],[]
best_methods = {'PM25_Concentration': 'akima', 
                'temperature': 'slinear',
                'humidity': 'slinear', 
                'wind_speed': 'slinear',
                'weather': 'nearest',
                'wind_direction': 'nearest'}
for station in beijing_met.station_id.unique():
    tmp_df = beijing_met[beijing_met.station_id == station]
    for cols in beijing_met.columns:
        try:
            tmp_df[cols] = tmp_df[cols].interpolate(best_methods[cols]).ffill().bfill()
        except:
            continue
    station_wise_met_df.append(tmp_df)
    tmp_df = beijing_aq[beijing_aq.station_id == station]
    for cols in beijing_aq.columns:
        try:
            tmp_df[cols] = tmp_df[cols].interpolate(best_methods[cols]).ffill().bfill()
        except:
            continue
    station_wise_aq_df.append(tmp_df)
    
filled_df_met = pd.concat(station_wise_met_df)
filled_df_aq = pd.concat(station_wise_aq_df)

merged_aq_met = pd.merge(filled_df_aq,filled_df_met)
display(merged_aq_met)


,time,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,1001,138.000000,159.4,56.3,0.9,50.8,17.2,0.0,20.0,56.0,7.92,13.0
1,2014-05-01 01:00:00,1001,124.000000,163.9,38.7,0.9,51.1,17.9,0.0,18.0,64.0,7.56,13.0
2,2014-05-01 02:00:00,1001,127.000000,148.4,55.6,1.0,27.2,16.6,0.0,18.0,70.0,5.76,13.0
3,2014-05-01 03:00:00,1001,129.000000,145.6,65.7,1.0,9.7,16.7,0.0,17.0,74.0,6.12,13.0
4,2014-05-01 04:00:00,1001,119.000000,119.3,66.9,1.0,2.0,16.5,0.0,17.0,75.0,4.68,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271555,2015-04-30 19:00:00,1036,77.000000,104.6,15.5,0.5,180.7,18.3,5.0,25.2,48.0,2.60,23.0
271556,2015-04-30 20:00:00,1036,94.000000,141.2,26.1,0.6,146.1,16.9,5.0,25.0,47.0,3.50,23.0
271557,2015-04-30 21:00:00,1036,96.064103,NaN,NaN,NaN,NaN,NaN,5.0,24.7,47.0,3.10,23.0
271558,2015-04-30 22:00:00,1036,88.000000,NaN,23.0,0.7,120.6,15.8,5.0,24.4,47.0,2.70,23.0
